WORKSHOP FINALE
===============

In [2]:
from pyplasm import *
import math
import csv

from larlib import *

In [13]:
def perimetro(): 
    with open("/home/gianlorenzo/PycharmProjects/ggpl/2017-12-11/perimetro.lines", "rb") as file:
        file_reader = csv.reader(file, delimiter=",")
        chiavi = {} ### diizionario delle chiavi
        nodi = [] ### array con i vertici
        archi= [] ### array con gli archi, ovvero gli spigoli
        i = 0 ### inizializzo un contatore
    
        for righe in file_reader:
            riga0 = math.floor(float(righe[0])*100)/100
            riga1 = math.floor(float(righe[1])*100)/100
            riga2 = math.floor(float(righe[2])*100)/100
            riga3 = math.floor(float(righe[3])*100)/100
        
            if not(riga0,riga1) in chiavi.keys():
                chiavi[(riga0,riga1)] = i
                nodi.append((riga0,riga1))
                i = i+1
            
            if not(riga2,riga3) in chiavi.keys():
                chiavi[(riga2,riga3)] = i
                nodi.append((riga2,riga3))
                i = i+1
            archi.append((chiavi[(riga0,riga1)],(chiavi[(riga2,riga3)])))
        pavimento = T(2)(450)(CUBOID([850,400]))
        corpoVilla = MULTEXTRUDE(OFFSET([20, 20])(STRUCT(MKPOLS([nodi,archi]))))(600)
    return TEXTURE("intonaco.jpg")(STRUCT([pavimento,corpoVilla]))

In [14]:
def tettoTriangolare1(depth,width):
    return TEXTURE("tetto.jpg")(S([1,2,3])([22,28.34,30])((ROTATE([1, 2])(PI)(ROTATE([2,3])(PI/2)(
        MULTEXTRUDE(MKPOL([[[0, 0],[width/2,width/4],[width,0]],[[1,2,3]],None]))(depth))))))

In [15]:
def sopra():
    return STRUCT([tettoTriangolare1(10,22)])


In [16]:
def comignoli():
    
    base = TEXTURE("intonaco.jpg")(T([1,2])([2.5,2.5])(CUBOID([10,10,20])))
    
    tettino = TEXTURE("intonaco.jpg")(T(3)(20)(CUBOID([15,15,2])))
    
    rialzo = TEXTURE("intonaco.jpg")(CUBOID([2,1.5,2]))
    
    rialziDestra = TEXTURE("intonaco.jpg")(
        STRUCT([T([1,2,3])([1,1,22])(rialzo),T([1,2,3])([1,6.75,22])(rialzo),T([1,2,3])([1,12.5,22])(rialzo)]))
    
    rialziSinistra = TEXTURE("intonaco.jpg")(
        STRUCT([T([1,2,3])([12,1,22])(rialzo),T([1,2,3])([12,6.75,22])(rialzo),T([1,2,3])([12,12,22])(rialzo)]))
    
    tetto = TEXTURE("intonaco.jpg")(T(3)(24)(CUBOID([15,15,1])))
    
    comignolo = S([1,2,3])([4,4,4])(STRUCT([base,tettino,rialziDestra,rialziSinistra,tettino,tetto]))
    
    return STRUCT([TEXTURE("intonaco.jpg")(comignolo)])

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f6697073ed0> >

In [18]:
def arch(lateral_width, height, deep):

    _lateral_width = float(lateral_width)
    _height = float(height)
    _deep = float(deep)

    def bottomArc(d):
        return BEZIER(S1)([[0, 0], [0, 2 * d / 3], [d, 2 * d / 3], [d, 0]])

    def topArc(d):
        return BEZIER(S1)([[0, 2 * d / 3], [d, 2 * d / 3]])

    def arc2D(d):
        return BEZIER(S2)([bottomArc(d), topArc(d)])

    def arc3D(d):
        def arc3D1(w):
            arco = arc2D(d)
            dominio = PROD([INTERVALS(1)(8), INTERVALS(1)(1)])
            ar = MAP(arco)(dominio)
            domin = PROD([ar, QUOTE([1.5])])
            return COMP([T(2)(w), R([2, 3])(PI / 2)])(domin)

        return arc3D1

    def Interarc(d1, d2):
        def Interarc1(w):
            return CUBOID([d1, w, 2 * d2 / 3])

        return Interarc1

    def Xarc(d1, d2):
        def Xarc1(w):
            return RIGHT([RIGHT([Interarc(d1, d2)(w), arc3D(d2)(w)]), Interarc(d1, d2)(w)])

        return Xarc1

    return Xarc(_lateral_width, _height)(_deep)

In [19]:
def porticato():
    return STRUCT([CUBOID([540,650,1800]),T(1)(2040)(CUBOID([540,650,1800]))])

def arcate():
    return STRUCT([(S(1)(0.15)(T([1,2])([4230,750])(arch(15,2700,-50)))),
                  S(1)(0.2)(T([1,2])([5130,750])(arch(15,2700,-50))),
                  S(1)(0.2)(T([1,2])([7750,750])(arch(85,2700,-50)))])

def porticatoCompleto():
    return STRUCT([T(2)(700)(porticato()),T(1)(-90)(arcate())])

In [20]:
VIEW(STRUCT([porticatoCompleto(),TOP([S([1,2,3])([3,3,3])(perimetro()),TOP([TEXTURE("tetto.jpg")(CUBOID([850,400])),
                                   LEFT([comignoli(),RIGHT([comignoli(),STRUCT([tettoTriangolare1(10,22)])])])])])]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f66973c39f0> >

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f982c208f90> >